## Advent of code 2020 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment
# to activate it from a git bash shell: source .venv/Scripts/activate
# to generate its requirements: pip freeze > .venv-requirements.txt
# to re-install from requirements: pip install -r .venv-requirements.txt

import collections
import itertools
import re
import copy
import math

In [ ]:
# utility functions

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

In [ ]:
# 2020 day 10
# mv ~/Downloads/input data_src/2020-day-10-input.txt
# idea: part 1 parse to numbers, then find device built-in adapter rating, then sort values and use them starting from 0, 
#  recording the 1-jolt and 3-jolt steps
# part 2: now you can apparently skip adapters, every adapter can be either in or out, there are apparently no doubles,
#  now generate all these combinations depth-first and count the valid ones, however that would take very long,
#  so split the sequence at every 3-jolt!

sample1='''
28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3
'''

sample2='''
16
10
15
5
1
11
7
19
6
12
4
'''

def split3(sordata): # split at differences of 3 into multiple lists
    res=[]
    group=[]
    prev=None
    for num in sordata:
        if prev is not None and abs(num-prev)>=3: # this is start of new group
            res.append(group)
            group=[]            
        group.append(num)
        prev=num
    if len(group)>0:
        res.append(group)
    return res

def count_arrang(sordata, pos, hist=None): # from pos in sorted data, try to reach last position,
    # possibly skipping adapters, return count (but count only when you reach the end)
    if pos==len(sordata)-1:
        if hist is not None:
            print(f'{hist=}')
        return 1
    adap=sordata[pos]
    # either continue with the next one or the ones after
    count=0
    for nexti in [pos+1, pos+2, pos+3]:
        if nexti>=len(sordata):
            break
        nextadap=sordata[nexti]
        diff=nextadap-adap
        if diff>=1 and diff<=3:
            hist2=hist
            if hist2 is not None:
                hist2=copy.deepcopy(hist)
                hist2.append(nextadap)
                print(f'{hist2=}')
            count+=count_arrang(sordata, nexti, hist=hist2)
    return count

sample1=open('data_src/2020-day-10-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[int(s) for s in lines]
data=data0
adapter_rating=max(data)+3
sordata=[0]
sordata.extend(sorted(data))
sordata.append(adapter_rating)
#print(f'{sordata=}')
sorgroups=split3(sordata)
#print(f'{sorgroups=}')
sorcounts=[ count_arrang(group, 0, hist=None) for group in sorgroups ]
print(f'{sorcounts=}')
print(f'{math.prod(sorcounts)=}')

# 1973822685184

In [ ]:
# 2020 day 9
# mv ~/Downloads/input data_src/2020-day-9-input.txt
# idea: part 1 note flex window/preamble size, parse to numbers, now use sliding window to check valid,
#  in the window can just generate set of sums of each pair

sample1='''
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
'''

def get_sums(win): # from list win generate pairs of different items, then their sums and return as set
    sums=set()
    for i in win:
        for j in win:
            if i!=j:
                sums.add(i+j)
    return sums

def find_sum(data, target): # from list find series summing to target, return series
    for i in range(len(data)): # start of series to try
        total=data[i]
        if total>=target: # must be series of at least 2
            continue
        for j in range(i+1, len(data)):
            total+=data[j]
            if total==target:
                return data[i:j+1]
            elif total>target:
                break
    return None

sample1=open('data_src/2020-day-9-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[int(s) for s in lines]
data=data0
window=25
for i in range(window, len(data)):
    num=data[i]
    win=data[i-window:i]
    sums=get_sums(win)
    #print(f'{num=} {win=} {sums=}')
    if num not in sums:
        print(f'found {num=}')
        sum_series=sorted(find_sum(data, num))
        print(f'{sum_series=}')
        sum_sum=sum_series[0]+sum_series[-1]
        print(f'{sum_sum=}')
        break
print(f'done')

In [ ]:
# 2020 day 8
# mv ~/Downloads/input data_src/2020-day-8-input.txt
# idea: part 1 simply parse the instructions to lists, with a third entry added for the run count of that instruction (0),
#  then just simulate with the instruction index and accumulator
# part 2: functionize, try

sample1='''
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
'''

sample1=open('data_src/2020-day-8-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
program=[ s.split() for s in lines ]
program0=[ [cmd, int(num), 0] for cmd, num in program ]

def run(flip_i=None): # runs a copy of program0, optionally with instruction i flipped nop/jmp, 
    # returns None if loops, return accum value if terminates ok
    program=copy.deepcopy(program0)
    if flip_i is not None:
        cmd, num, runs=program[flip_i]
        if cmd=='jmp':
            program[flip_i][0]='nop'
        elif cmd=='nop':
            program[flip_i][0]='jmp'
    instr=0
    accum=0
    while True:
        cmd, num, runs=program[instr]
        if runs>0: # already visited
            #print(f'{accum=}')
            return None
        program[instr][2]=runs+1
        match cmd:
            case 'acc':
                accum+=num
                instr+=1
            case 'jmp':
                instr+=num
            case 'nop':
                instr+=1
            case _:
                assert False
        if instr==len(program):
            return accum

for i in range(len(program)):
    res=run(flip_i=i)
    if res is not None:
        print(f'{res=}')
        break
print(f'done')
    

In [ ]:
# 2020 day 7
# mv ~/Downloads/input data_src/2020-day-7-input.txt
# idea: part 1 first parse rules into a dict - bag name e.g. light red to another dict, 'bright white' to 1,
#  'muted yellow' to 2, now depth first search for reaching the target bag yes/no, and add the yesses

sample1='''
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
'''

sample2='''
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
'''

sample1=open('data_src/2020-day-7-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

def line_to_tuple(line):
    line=line.replace('bags', '').replace('bag', '').replace('.', '')
    parts=line.split('contain')
    bag1=parts[0].strip()
    parts=parts[1].split(',')
    dct={}
    for part in parts:
        part=part.strip()
        if result := re.match(r'(\d+)\s*([\w\s]+)', part):
            key=result.group(2)
            num=int(result.group(1))
            dct[key]=num
        elif part=='no other':
            pass
        else:
            assert False
    #print(f'{bag1=} {parts=} {dct=}')
    return (bag1, dct)

rules=dict([ line_to_tuple(line) for line in lines ])

def bag_search(name, target): # is target reachable from name?
    rule=rules[name]
    for name2 in rule.keys():
        if name2==target:
            return True
        if bag_search(name2, target):
            return True
    return False

def bag_count(name): # how many bags below this one, including itself?
    count=1
    rule=rules[name]
    for name2,num in rule.items():
        count+=num*bag_count(name2)
    return count

#print(f'{rules=}')
#count=0
#for topname in rules.keys():
#    if bag_search(topname, 'shiny gold'):
#        count+=1
#    #print(f'{topname=} {count=}')
count=bag_count('shiny gold')-1
print(f'{count=}')

In [ ]:
# 2020 day 6
# mv ~/Downloads/input data_src/2020-day-6-input.txt
# idea: part 1 per group collect all chars, put in set, add lengths, part 2 slight modification

sample1='''
abc

a
b
c

ab
ac

a
a
a
a

b
'''

sample1=open('data_src/2020-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() ] #if len(s)>0 ]
groups=get_line_groups(lines)

def get_group_score(group):
    common=None
    for line in group:
        chars=set()
        for c in line:
            chars.add(c)
        if common is None:
            common=chars
        else:
            common=common & chars
    return len(common)

gscores=[get_group_score(group) for group in groups]
#gscores
sum(gscores)

In [ ]:
# 2020 day 5
# mv ~/Downloads/input data_src/2020-day-5-input.txt
# idea: n/a

sample1='''
BFFFBBFRRR
FFFBBBFRRR
BBFFBBFRLL
'''

sample1=open('data_src/2020-day-5-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
seatmax=-1
seats=set()
for line in lines:
    line=line.strip()
    rowlet=line[:7]
    rowlet=rowlet.replace('F', '0').replace('B', '1')
    collet=line[7:]
    collet=collet.replace('L', '0').replace('R', '1')
    rownum=int(rowlet, 2)
    colnum=int(collet, 2)
    seatid=rownum*8+colnum
    #print(f'{line=} {rowlet=} {rownum=} {collet=} {colnum=} {seatid=}')
    if seatid>seatmax:
        seatmax=seatid
    seats.add(seatid)
print(f'{seatmax=}')
seats=list(sorted(seats))
last= -1
for seatid in seats: # find seat with one before it missing, that's yours
    if last>=0 and last!=seatid-1:
        print(f'your seat: {seatid-1}')
        break
    last=seatid

In [ ]:
# 2020 day 4 part 1
# mv ~/Downloads/input data_src/2020-day-4-input.txt
# idea for part 2: add a function to validate value based on code and val, also add samples to test

sample1='''
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
''' # 2 valid

sample2='''
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
''' # 0 valid

sample3='''
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
''' # 4 valid

sample1=open('data_src/2020-day-4-input.txt').read()
lines=[s for s in sample1.splitlines() ] # if len(s)>0 
groups=get_line_groups(lines)
#print(f'{groups=}')

def validate_value(code, val):
    assert val
    match code:
        case 'byr':
            if not (result := re.fullmatch(r'(\d{4})', val)):
                return False
            num=int(result.group(1))
            return num>=1920 and num<=2002
        case 'iyr':
            if not (result := re.fullmatch(r'(\d{4})', val)):
                return False
            num=int(result.group(1))
            return num>=2010 and num<=2020
        case 'eyr':
            if not (result := re.fullmatch(r'(\d{4})', val)):
                return False
            num=int(result.group(1))
            return num>=2020 and num<=2030
        case 'hgt':
            if not (result := re.fullmatch(r'(\d+)(\w+)', val)):
                return False
            num=int(result.group(1))
            if result.group(2)=='cm':
                return num>=150 and num<=193
            elif result.group(2)=='in':
                return num>=59 and num<=76
            else:
                return False
        case 'hcl':
            if not (result := re.fullmatch(r'\#[0-9a-f]{6}', val)):
                return False
            return True
        case 'ecl':
            return val in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}
        case 'pid':
            if not (result := re.fullmatch(r'(\d{9})', val)):
                return False
            return True
        case 'cid':
            return True
        case _:
            assert False

# main
count_valid=0
for group in groups:
    fields=set()
    for line in group:
        parts=line.split()
        for part in parts:
            if part:
                code,val=part.split(':')
                if validate_value(code, val):
                    fields.add(code)
    required={'byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'}
    reqmiss=required-fields
    #print(f'{fields=} {reqmiss=}')
    if len(reqmiss)<1:
        count_valid+=1
print(f'{count_valid=}')

# part 1: 2 and 230
# part 2: 156

In [ ]:
# 2020 day 3
# mv ~/Downloads/input data_src/2020-day-3-input.txt
# idea: just go down, x=(x+3)%len(row), also: first spot isn't a tree in either input

sample1='''
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
'''

sample1=open('data_src/2020-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
print(f'{len(lines)=} {len(lines[0])=}')

def get_trees(dx, dy):
    count=0
    row=0
    col=0
    while row<len(lines):
        if lines[row][col] == '#':
            count+=1
        col=(col+dx) % len(lines[row])
        row+=dy
        #if row<100:
        #    print(f'{count=} {row=} {col=}')
    return count

mult=1
for dx, dy in [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]:
    count=get_trees(dx, dy)
    mult *= count
    print(f'{dx=} {dy} {count=} {mult=}')


In [ ]:
# 2020 day 2 part one
# mv ~/Downloads/input data_src/2020-day-2-input.txt

sample1='''
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
'''

sample1=open('data_src/2020-day-2-input.txt').read()
tups=[ result.group(1, 2, 3, 4) for s in sample1.splitlines() if (result:= re.match(r'(\d+)-(\d+)\s+(\w):\s*(\w+)', s)) ]
#print(f'{tups}')
numvalid=0
for mino, maxo, ch, word in tups:
    count=collections.Counter() # count of all chars in word
    for c in word:
        count[c] += 1
    valid= (count[ch] >= int(mino) and count[ch] <= int(maxo))
    #print(f'{word}: {valid}')
    if valid:
        numvalid+=1
print(f'{numvalid}')

In [ ]:
# 2020 day 2 part two

numvalid=0
for mino, maxo, ch, word in tups:
    count=0
    if word[int(mino)-1] == ch:
        count+=1 
    if word[int(maxo)-1] == ch:
        count+=1 
    valid= (count==1)
    #print(f'{word}: {valid}')
    if valid:
        numvalid+=1
print(f'{numvalid}')

In [ ]:
# 2020 day 1
# 'curl https://adventofcode.com/2020/day/1/input --output data_src/2020-day-1-input.txt'

sample1='''
1721
979
366
299
675
1456
'''

sample1=open('data_src/2020-day-1-input.txt').read()
nums=[int(s) for s in sample1.splitlines() if len(s)>0 ]
combis=list(itertools.combinations(nums, 3)) # 2 for first half of the puzzle, 3 for second part
#[ x*y for x,y in combis if x+y==2020 ] # first half
[ x*y*z for x,y,z in combis if x+y+z==2020 ] # second half

### Lessons learned for competing in the Advent of Code
* It's a grind to win, so get up 10-15 minutes early and be ready at 6:00 AM every day, with 2 glasses of water, music, etc. This is most crucial in the first days when many people can potentially finish quickly.
* Due to the scoring system if you want to be in the top, and likely will be competing with a small group of fanatics, if you 'take a break' one day and let 20 other people finish first you'll be set back 20 points that you can regain only very slowly as in the top you can gain only 1 or 2 points per day on the people above you (unless they take a break).
* VS Code with python and a jupyter notebook feels like very much the right tool for the job
* First focus on getting your input in the best shape, don't rush through this as it will only take a little time anyway
* First few days the problems are so simple you likely don't need functions
* Afterwards split the problem into 3-5 smaller functions that are easy to write, each with a short comment to describe what it does, this will often speed up part 2 a lot